# Project 1: iris classification with DNN - 20201514 이현수
- Keras를 사용하여 iris dataset 분류를 수행하고 accuracy를 출력합니다.
- 최대한 overfitting을 억제하면서 accuracy를 높일 수 있도록 합시다.
- data: ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'] (꽃받침의 길이, 꽃받침의 너비, 꽃잎의 길이, 꽃잎의 너비) (in centimeter)
- target: [0:'Iris-setosa', 1:'Iris-versicolor', 2:'Iris-virginica']

![](https://editor.analyticsvidhya.com/uploads/51518iris%20img1.png)

https://www.kaggle.com/datasets/uciml/iris

https://en.wikipedia.org/wiki/Iris_flower_data_set

# Requirements
- Dataset을 train, validation, test로 적절히 나눈 뒤, train + validation으로 하이퍼파라미터 최적화, test로 최종 accuracy를 출력합니다. (1)
- 적절한 layer 구성 (몇 개의 hidden layer을 사용할 것인지), activation function 설정 및 그에 따른 하이퍼파라미터 최적화, dropout 최적화 작업이 필요합니다. (4)
    * Hidden layer의 노드 개수는 대체로 input feature의 수와 output node의 수의 사이값이 적절한 것으로 알려져 있습니다.
- 전처리 과정이 필요한 경우 Keras layering으로 구현합니다. (1)
- Callback을 사용하여 early stopping을 구현합니다. (1)

**최종 최적화 이후, model.summary() 출력 필요**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

print(X.shape, y.shape)
print(X[0], y[0])

# train/test 세트로 데이터를 나눔
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# train 세트를 다시 train/validation 세트로 데이터를 나누자
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# 데이터셋 크기 출력
print("Train set: ", X_train.shape)
print("Validation set: ", X_val.shape)
print("Test set: ", X_test.shape)

(150, 4) (150,)
[5.1 3.5 1.4 0.2] 0
Train set:  (90, 4)
Validation set:  (30, 4)
Test set:  (30, 4)


## Num of hidden layers, Activation Function optimization

In [3]:
from tensorflow import keras
optimization_arr = []

# 모델 구성
def optimization_model(hidden_layers=1, activation='relu', dropout=0.2):
    model = keras.Sequential()
    model.add(keras.layers.Rescaling(1/X.max(), input_shape=(4,)))
    model.add(keras.layers.Dense(units=100, activation=activation, input_shape=(4,), name='Hidden'))
    model.add(keras.layers.Dropout(dropout))
    
    # hidden layer 개수
    for i in range(hidden_layers-1):
        model.add(keras.layers.Dense(units=100, activation=activation))
        model.add(keras.layers.Dropout(dropout))
    
    # output layer는 softmax함수로 픽스
    model.add(keras.layers.Dense(units=10, activation='softmax', name='Output'))
    
    return model

# hidden layer 개수, activation 함수 Optimization
for num_of_hidden_layers in [1,2,3,4,5]:
    for activation_function in ['relu', 'sigmoid', 'tanh']:
        model = optimization_model(num_of_hidden_layers, activation_function)
        #model.summary()

        model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
        model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

        train_loss, train_acc = model.evaluate(X_train, y_train)
        val_loss, val_acc = model.evaluate(X_val, y_val)
        test_loss, test_acc = model.evaluate(X_test, y_test)
        optimization_arr.append([num_of_hidden_layers, activation_function, train_acc, val_acc, test_acc])

sorted_arr = sorted(optimization_arr, key=lambda x: x[4])

for i in range (len(sorted_arr)):
    print(sorted_arr[i])

Epoch 1/20
3/3 [==============================] - 1s 109ms/step - loss: 2.3176 - accuracy: 0.0444 - val_loss: 2.2186 - val_accuracy: 0.0000e+00
Epoch 2/20
3/3 [==============================] - 0s 18ms/step - loss: 2.2108 - accuracy: 0.0889 - val_loss: 2.1395 - val_accuracy: 0.4333
Epoch 3/20
3/3 [==============================] - 0s 21ms/step - loss: 2.1204 - accuracy: 0.3000 - val_loss: 2.0759 - val_accuracy: 0.4333
Epoch 4/20
3/3 [==============================] - 0s 19ms/step - loss: 2.0703 - accuracy: 0.3000 - val_loss: 2.0183 - val_accuracy: 0.4333
Epoch 5/20
3/3 [==============================] - 0s 18ms/step - loss: 2.0116 - accuracy: 0.3556 - val_loss: 1.9654 - val_accuracy: 0.4333
Epoch 6/20
3/3 [==============================] - 0s 20ms/step - loss: 1.9618 - accuracy: 0.3000 - val_loss: 1.9146 - val_accuracy: 0.4333
Epoch 7/20
3/3 [==============================] - 0s 21ms/step - loss: 1.9152 - accuracy: 0.3556 - val_loss: 1.8657 - val_accuracy: 0.4333
Epoch 8/20
3/3 [======

## Optimizer, Learning rate optimization
##### 위에서, hidden layer는 5개일때, Activation Function은 tanh일때 Accuracy가 가장 좋았습니다.
##### (accuracy가 1.0으로 나오는 값은 과접합으로 생각하고 제외했습니다.)

In [5]:
model = optimization_model(5, 'tanh')
lr_arr = []

for lr in [1, 0.1, 0.01, 0.001]:
    adagrad = keras.optimizers.Adagrad(learning_rate=lr) # 기본값은 lr=0.01
    adam = keras.optimizers.Adam(learning_rate=lr) # 기본값은 lr=0.001
    sgd = keras.optimizers.SGD(learning_rate=lr) # 기본값은 lr=0.01
    rmsprop = keras.optimizers.RMSprop(learning_rate=lr) # 기본값은 lr=0.001

    for optimizer in [adagrad, adam, sgd, rmsprop]:
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics='accuracy')
        model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
        train_loss, train_acc = model.evaluate(X_train, y_train)
        val_loss, val_acc = model.evaluate(X_val, y_val)
        test_loss, test_acc = model.evaluate(X_test, y_test)
        lr_arr.append([lr, optimizer, train_acc, val_acc, test_acc])
    
sorted_arr = sorted(lr_arr, key=lambda x: x[4])

for i in range (len(sorted_arr)):
    print(sorted_arr[i])

Epoch 1/20
3/3 [==============================] - 1s 136ms/step - loss: 5.2881 - accuracy: 0.2000 - val_loss: 18.1261 - val_accuracy: 0.4333
Epoch 2/20
3/3 [==============================] - 0s 21ms/step - loss: 39.7346 - accuracy: 0.3333 - val_loss: 5.8600 - val_accuracy: 0.4333
Epoch 3/20
3/3 [==============================] - 0s 33ms/step - loss: 23.3081 - accuracy: 0.3222 - val_loss: 5.3355 - val_accuracy: 0.4333
Epoch 4/20
3/3 [==============================] - 0s 21ms/step - loss: 16.4576 - accuracy: 0.3111 - val_loss: 3.5559 - val_accuracy: 0.4333
Epoch 5/20
3/3 [==============================] - 0s 22ms/step - loss: 10.4719 - accuracy: 0.4333 - val_loss: 16.1509 - val_accuracy: 0.4333
Epoch 6/20
3/3 [==============================] - 0s 20ms/step - loss: 28.1472 - accuracy: 0.2444 - val_loss: 9.6996 - val_accuracy: 0.4333
Epoch 7/20
3/3 [==============================] - 0s 20ms/step - loss: 13.6338 - accuracy: 0.4222 - val_loss: 4.5254 - val_accuracy: 0.4333
Epoch 8/20
3/3 [==

## Epoch optimization
##### 위에서, Optimizer는 RMSprop이, Learning Rate는 0.001에서 Accuracy가 가장 좋았습니다.

In [7]:
model = optimization_model(5, 'tanh')
epochs_arr = []

rmsprop = keras.optimizers.RMSprop(lr=0.001)
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

# epochs 설정 -> validation 성능과 연관이 깊기 때문에 validation set의 score에 따라 선택
for epochs in [10, 15, 20, 25, 30]:
    model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))
    val_loss, val_acc = model.evaluate(X_val, y_val)
    epochs_arr.append([epochs, val_loss, val_acc])

sorted_arr = sorted(epochs_arr, key=lambda x: x[2])

for i in range (len(sorted_arr)):
    print(sorted_arr[i])

Epoch 1/10
3/3 [==============================] - 1s 130ms/step - loss: 1.9087 - accuracy: 0.2444 - val_loss: 1.2902 - val_accuracy: 0.5333
Epoch 2/10
3/3 [==============================] - 0s 22ms/step - loss: 1.2400 - accuracy: 0.4222 - val_loss: 1.0673 - val_accuracy: 0.5667
Epoch 3/10
3/3 [==============================] - 0s 22ms/step - loss: 1.0367 - accuracy: 0.5889 - val_loss: 0.9700 - val_accuracy: 0.3667
Epoch 4/10
3/3 [==============================] - 0s 23ms/step - loss: 0.8579 - accuracy: 0.6333 - val_loss: 0.7175 - val_accuracy: 0.9333
Epoch 5/10
3/3 [==============================] - 0s 22ms/step - loss: 0.7276 - accuracy: 0.7111 - val_loss: 0.7813 - val_accuracy: 0.5667
Epoch 6/10
3/3 [==============================] - 0s 21ms/step - loss: 0.6951 - accuracy: 0.6444 - val_loss: 0.6034 - val_accuracy: 0.5667
Epoch 7/10
3/3 [==============================] - 0s 21ms/step - loss: 0.5520 - accuracy: 0.7889 - val_loss: 0.5782 - val_accuracy: 0.5667
Epoch 8/10
3/3 [==========

## Dropout optimization
##### 위에서, Epoch는 20일때 Accuracy가 가장 좋았습니다.
##### (validation의 loss가 가장 낮고, accuracy가 가장 높은 값으로 찾았습니다.) 

In [8]:
dropout_arr = []

for dropout in [0.1, 0.2, 0.3, 0.4, 0.5]:
    # Num of hidden layer & Active function 최적화 결과 -> 5, tanh
    model = optimization_model(5, 'tanh', dropout=dropout)

    # Optimizer & Learning Rate 최적화 결과 -> RMSprop, 0.001
    rmsprop = keras.optimizers.RMSprop(lr=0.001)
    model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

    # Epoch Optimization -> 20
    model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
    train_loss, train_acc = model.evaluate(X_train, y_train)
    val_loss, val_acc = model.evaluate(X_val, y_val)
    test_loss, test_acc = model.evaluate(X_test, y_test)
    dropout_arr.append([dropout, train_acc, val_acc, test_acc])

sorted_arr = sorted(dropout_arr, key=lambda x: x[3])

for i in range (len(sorted_arr)):
    print(sorted_arr[i])

Epoch 1/20
3/3 [==============================] - 1s 103ms/step - loss: 1.8945 - accuracy: 0.2444 - val_loss: 1.1830 - val_accuracy: 0.4333
Epoch 2/20
3/3 [==============================] - 0s 20ms/step - loss: 1.2361 - accuracy: 0.4111 - val_loss: 1.0771 - val_accuracy: 0.2667
Epoch 3/20
3/3 [==============================] - 0s 20ms/step - loss: 0.9284 - accuracy: 0.6444 - val_loss: 0.7958 - val_accuracy: 0.9000
Epoch 4/20
3/3 [==============================] - 0s 21ms/step - loss: 0.7767 - accuracy: 0.6889 - val_loss: 0.6435 - val_accuracy: 0.9333
Epoch 5/20
3/3 [==============================] - 0s 20ms/step - loss: 0.6488 - accuracy: 0.7111 - val_loss: 0.7912 - val_accuracy: 0.5667
Epoch 6/20
3/3 [==============================] - 0s 21ms/step - loss: 0.5685 - accuracy: 0.7556 - val_loss: 0.4936 - val_accuracy: 0.9333
Epoch 7/20
3/3 [==============================] - 0s 19ms/step - loss: 0.4803 - accuracy: 0.7778 - val_loss: 0.4423 - val_accuracy: 0.9000
Epoch 8/20
3/3 [==========

## Summary: 
### Num of hidden layers = 5 
### Activation function = tanh 
### Optimizer, Learning rate = RMSprop, 0.001 
### Epoch, Dropout prob = 20, 0.2
## Final evaluation with early stopping

In [9]:
# Num of hidden layer & Active function 최적화 결과 -> 5, tanh
model = optimization_model(5, 'tanh', dropout=0.2)

# Optimizer & Learning Rate 최적화 결과 -> RMSprop, 0.001
rmsprop = keras.optimizers.RMSprop(lr=0.001)
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

# Checkpoint, Earlystopping
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5', 
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,
                                                  restore_best_weights=True)

# Epoch Optimization -> 20
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

print('Early Stopping : ', early_stopping_cb.stopped_epoch)
model.summary()

train_loss, train_acc = model.evaluate(X_train, y_train)
val_loss, val_acc = model.evaluate(X_val, y_val)
test_loss, test_acc = model.evaluate(X_test, y_test)
print('train accuracy is ', train_acc)
print('validation accuracy is ', val_acc)
print('test accuracy is ', test_acc)

Epoch 1/20
3/3 [==============================] - 1s 135ms/step - loss: 1.8829 - accuracy: 0.2000 - val_loss: 1.4094 - val_accuracy: 0.4000
Epoch 2/20
3/3 [==============================] - 0s 52ms/step - loss: 1.2359 - accuracy: 0.4667 - val_loss: 1.1530 - val_accuracy: 0.5667
Epoch 3/20
3/3 [==============================] - 0s 50ms/step - loss: 1.0718 - accuracy: 0.5667 - val_loss: 0.9723 - val_accuracy: 0.5667
Epoch 4/20
3/3 [==============================] - 0s 46ms/step - loss: 0.9689 - accuracy: 0.5556 - val_loss: 0.9041 - val_accuracy: 0.5000
Epoch 5/20
3/3 [==============================] - 0s 48ms/step - loss: 0.9147 - accuracy: 0.6333 - val_loss: 0.6836 - val_accuracy: 0.7333
Epoch 6/20
3/3 [==============================] - 0s 52ms/step - loss: 0.7555 - accuracy: 0.6889 - val_loss: 0.6143 - val_accuracy: 0.7333
Epoch 7/20
3/3 [==============================] - 0s 54ms/step - loss: 0.6661 - accuracy: 0.7333 - val_loss: 0.5239 - val_accuracy: 0.7333
Epoch 8/20
3/3 [==========